In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import glob

# We should consider filtering our data a bit because it's a bit messy currently

**I've written this scaffold to make things easier but code similar to what I've written below should be written as a function or two and run for every host and every virus probably saving as a new "tsv" file called "yadayada.clean.tsv"**

**This currently works on E. coli but it's possible that there will be some weird genomes that something in here screws up on so we'll need to think about double checking things to make sure that we never remove too much data. Which is to say that df.shape shouldn't change *that* dramatically from beginning to end and if it does it's likely that there is a bug in the code that needs to be investigated**

In [3]:
###Read in the starting TSV
df = pd.read_csv('../Data/host_genomes/562.tsv', sep='\t', index_col=0)
###Reset the index so that it runs from zero to whatever
df = df.reset_index(drop=True)
print(df.shape)
df.head()
df["qualifiers"][1]
#df.head()

(4379, 12)


'ID=cds-NP_414543.1;Parent=gene-b0002;Dbxref=UniProtKB/Swiss-Prot:P00561,Genbank:NP_414543.1,ASAP:ABE-0000008,ECOCYC:EG10998,EcoGene:EG10998,GeneID:945803;Name=NP_414543.1;gbkey=CDS;gene=thrA;locus_tag=b0002;orig_transcript_id=gnl|b0002|mrna.b0002;product=fused aspartate kinase/homoserine dehydrogenase 1;protein_id=NP_414543.1;transl_table=11'

**First, make sure the table only uses each `locus_tag` once. These should be unique identifiers for the hosts and should therefore only appear once. If they appear more than once I have no idea what's happening. The virus dataframes won't be able to run this part of the code, however.**

In [4]:
df['locus_tag'] = df['qualifiers'].str.split('Parent=gene-', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]

In [5]:
test_string = df.loc[0]["qualifiers"]

In [6]:
test_string

'ID=cds-NP_414542.1;Parent=gene-b0001;Dbxref=UniProtKB/Swiss-Prot:P0AD86,Genbank:NP_414542.1,ASAP:ABE-0000006,ECOCYC:EG11277,EcoGene:EG11277,GeneID:944742;Name=NP_414542.1;gbkey=CDS;gene=thrL;locus_tag=b0001;orig_transcript_id=gnl|b0001|mrna.b0001;product=thr operon leader peptide;protein_id=NP_414542.1;transl_table=11'

In [7]:
df['locus_tag'].value_counts()[:10]

b0470    3
b4623    3
b4492    3
b0484    3
b4579    2
b4580    2
b4587    2
b3443    2
b4569    2
b4571    2
Name: locus_tag, dtype: int64

**Oooof, let's get rid of those**

In [8]:
df = df.drop_duplicates(subset=['locus_tag'], keep=False)
print(df.shape)

(4335, 13)


In [9]:
df['locus_tag'].value_counts()[:10]

b2663    1
b3882    1
b1365    1
b1483    1
b1914    1
b4300    1
b3404    1
b0546    1
b4342    1
b3344    1
Name: locus_tag, dtype: int64

**Voila!**

**How about any weird length genes? This should be checked and removed for all genomes (host and virus)**

In [28]:
((df['stop']-df['start']) %3).value_counts()

2    4305
1      22
0       8
dtype: int64

**Clean them up too**

In [29]:
df = df[(df['stop']-df['start']) %3 == 2]
print(df.shape)

(4305, 13)


**Remove phage/prophage genes from descriptors. This again should only be run for the host genomes so you can see maybe we'll want separate functions like "def clean_host(...)" and "def clean_virus(...)"**

In [30]:
filter_word = 'phage'
# filter_word = 'phage'
virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
print(virus_genes.shape)

(246, 13)


In [31]:
df = df[(df['qualifiers'].str.contains(filter_word)==False)]
df.shape

(4059, 13)

0.9676860934795153

In [33]:
virus_genes.loc[93]["qualifiers"]

'ID=cds-WP_011000059.1;Parent=gene-RS_RS00465;Dbxref=Genbank:WP_011000059.1,GeneID:1218895;Name=WP_011000059.1;gbkey=CDS;inference=COORDINATES: similar to AA sequence:RefSeq:WP_003261510.1;locus_tag=RS_RS00465;product=phage holin family protein;protein_id=WP_011000059.1;transl_table=11'

# Add in knowledge of codon usage bias

** This is an important control and you'll have to install this "iCUB" software to make this run. I worked on this project with a former graduate student in my PhD lab and the code is here: https://github.com/amarallab/iCUB **

**The major bug that I can anticipate cropping up is if the organism uses a "non-standard" genetic code but we'll cross that bridge when we get there. This will also probably cause an error if any "non-standard" nucleotides appear in a given genome which will almost certainly occur and need to be fixed**

In [22]:
import sys
sys.path.append('/Users/ChaseWeaver/Desktop/Projects/iCUB-master/')
import iCUB

In [23]:
for index in df.index:
    nt_seq = df.at[index, "coding_sequence"]
    if len(nt_seq)%3 == 0:
        df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()

**If our genes are clean and good then we shouldn't have any null values in either of these categories and we can then write this new dataframe**

In [24]:
print(df[df['iCUB'].isnull()==True].shape)
print(df[df['energy_binding'].isnull()==True].shape)

(0, 13)
(0, 13)


# Final notes:

**As I mentioned, we aren't going to want to step through this code for every genome so your task is to write this up into some functions that you can then put into for loops and run for everyone. I'm imagining something like:**

<code>
for host_tsv_file in glob.glob('blahblah/\*.tsv): 
    clean_host_tsv(host_tsv_file)
</code>
    
**where `clean_host_tsv()` will read the tsv, clean it up, check that no problems occurred, and write the new tsv. Something along the lines of:**

<code>
def clean_host_tsv(tsv_location):

    df = pd.read_tsv(tsv_location...)
    starting_shape = df.shape
    #Code that does the work goes here
    ending_shape = df.shape
    #Code that compares ending and starting shapes to make sure things look reasonable goes here
    df.to_csv(tsv_location.replace('.tsv', '.clean.tsv'))
</code>


**And then again we'll have something like this for the viruses:**

<code>
for virus_folder in glob.glob('blahblah/\*\_rep\_viruses/'):
    for virus_tsv_file in glob.glob(virus_folder + '\*.tsv'):
        clean_virus_tsv(virus_tsv_file)
</code>


In [ ]:
df = pd.read_csv('../Data/host_genomes/562.tsv', sep = "\t", index_col = 0)
print(df.shape)

In [10]:
def clean_host_tsv(df):
    """
    
    """
    
    initial_shape = df.shape
    df = df.reset_index(drop = True)
    
    #Ensure each locus tag is only used once
    df['locus_tag'] = df['qualifiers'].str.split('locus_tag=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ["locus_tag"], keep = False)
    
    #ensure that only genes with a total length divisible by 3 are used
    df = df[(df['stop']-df['start']) %3 == 2]
    
    #filter out prophages from dataframe
    filter_word = 'phage'
    virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
    df = df[(df['qualifiers'].str.contains(filter_word)==False)]
    
    #adjust for codon bias
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq)%3 == 0:
            df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()
    
    
    
    
    
    return df

In [16]:
# for tsv_location in glob.glob("../Data/host_genomes/*.tsv"):
#     print(tsv_location)

In [35]:
for tsv_location in glob.glob("../Data/host_genomes/*.tsv"):
    if "clean" in tsv_location:
        continue
    df = pd.read_csv(tsv_location, sep = "\t", index_col = 0)
    initial_shape = df.shape
    df = clean_host_tsv(df)
    final_shape = df.shape
    gene_ratio = final_shape[0]/initial_shape[0]
    if gene_ratio <= .8 or gene_ratio == 1:
        print(tsv_location)
        break
    
    clean_tsv_loc = tsv_location.replace(".tsv",".clean.tsv")
    df.to_csv(clean_tsv_loc, sep = "\t")
    


In [15]:
#df['qualifiers'].str.split('ID=')

In [41]:
tsv_location ="../Data/host_genomes/717959.tsv"
     
df = pd.read_csv(tsv_location, sep = "\t", index_col = 0)
initial_shape = df.shape
df = clean_host_tsv(df)
final_shape = df.shape
gene_ratio = final_shape[0]/initial_shape[0]
if gene_ratio <= .8 or gene_ratio == 1:
    print(tsv_location)
    #break

clean_tsv_loc = tsv_location.replace(".tsv",".clean.tsv")
df.to_csv(clean_tsv_loc, sep = "\t")

In [31]:
print(gene_ratio)

0.9971290148932352


In [13]:
df['viral_id'] = df['qualifiers'].str.split('ID=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]

In [29]:
#df["viral_id"]

In [12]:
def clean_virus_tsv(df):
    """
    
    """
    
    initial_shape = df.shape
    df = df.reset_index(drop = True)
    
    #Ensure each locus tag is only used once
    df['viral_id'] = df['qualifiers'].str.split('ID=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ["viral_id"], keep = False)
    
    #ensure that only genes with a total length divisible by 3 are used
    df = df[(df['stop']-df['start']) %3 == 2]
    
    #filter out prophages from dataframe
#     filter_word = 'phage'
#     virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
#     df = df[(df['qualifiers'].str.contains(filter_word)==False)]
    
    #adjust for codon bias
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq)%3 == 0:
            df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()
    
    return df

In [31]:
for virus_folder in glob.glob('../Data/*_rep_viruses'):
    for virus_tsv_file in glob.glob(virus_folder + '*.tsv'):
        
        if "clean" in virus_tsv_file:
            continue
        df = pd.read_csv(virus_tsv_file, sep = "\t", index_col = 0)
        
        initial_shape = df.shape
        
        df = clean_virus_tsv(virus_tsv_file)
        final_shape = df.shape
#         gene_ratio = final_shape[0]/initial_shape[0]
#         if gene_ratio <= .8 or gene_ratio == 1:
#             print(tsv_location)
#             break
    
        clean_tsv_loc = tsv_location.replace(".tsv",".clean.tsv")
        df.to_csv(clean_tsv_loc, sep = "\t")

In [26]:
tsv_location ="../Data/287_rep_viruses/32.tsv"
     
df = pd.read_csv(tsv_location, sep = "\t", index_col = 0)
initial_shape = df.shape
df = clean_virus_tsv(df)
final_shape = df.shape
# gene_ratio = final_shape[0]/initial_shape[0]
# if gene_ratio <= .8 or gene_ratio == 1:
#     print(tsv_location)
#     #break

clean_tsv_loc = tsv_location.replace(".tsv",".clean.tsv")
df.to_csv(clean_tsv_loc, sep = "\t")

In [27]:
initial_shape

(291, 12)

In [28]:
final_shape

(291, 14)